<a href="https://colab.research.google.com/github/mstagomori/emd-desafio-junior-data-scientist/blob/main/respostas/analise_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Desafio Técnico em Python - API

## Importando APIs e Bibliotecas

In [103]:
import pandas as pd
import numpy as np
import requests
import json
import plotly.express as px
import datetime
import calendar

# Open-Meteo Historical Weather API

!pip install openmeteo-requests -q
!pip install requests-cache retry-requests numpy pandas -q
import openmeteo_requests
import requests_cache
from retry_requests import retry

In [104]:
# Desativa SettingWarning
pd.options.mode.chained_assignment = None  # default='warn'

## Utilitários

In [105]:
# Plota gráfico de barras
def plot_bar(data, x, y, color, text, title, subtitle, labels, color_discrete_sequence=None, orientation='v'):
  fig = px.bar(data,
               x = x,
               y = y,
               color = color,
               text = text,
               title=title + f' <br><sup>{subtitle}</sup>',
               labels=labels,
               orientation=orientation,
               color_discrete_sequence=color_discrete_sequence)
  fig.update_traces(textposition='outside', showlegend=False)
  fig.update_layout(width=1080, bargap=0.2)
  fig.show()

In [106]:
# Cores para visualização
BLUE_HIGHLIGHT = "#3d87ff"
LIGHT_GRAY = "#9b9b9b"

## Pergunta 1: Quantos feriados há no Brasil em todo o ano de 2024?

In [107]:
response = requests.get('https://date.nager.at/api/v3/publicholidays/2024/BR')
feriados = json.loads(response.content)

print(f'Há {len(feriados)} feriados no Brasil em todo o ano de 2024')

Há 14 feriados no Brasil em todo o ano de 2024


## Pergunta 2: Qual mês de 2024 tem o maior número de feriados?

In [108]:
def get_month(date):
  return date.split('-')[1]

def get_holidays_by_month(feriados):
  feriados_por_mes = {}
  for feriado in feriados:
    mes = calendar.month_name[int(get_month(feriado['date']))]
    if mes in feriados_por_mes:
      feriados_por_mes[mes] += 1
    else:
      feriados_por_mes[mes] = 1
  return feriados_por_mes



feriados_por_mes = get_holidays_by_month(feriados)

df_feriados_por_mes = pd.DataFrame(list(feriados_por_mes.items()), columns=['mes', 'feriados'])

plot_bar(data=df_feriados_por_mes,
         x='mes',
         y='feriados',
         color='mes',
         text='feriados',
         title='Feriados por Mês 2024',
         subtitle='Fevereiro, Março, Maio e Novembro são os meses com mais feriados em 2024',
         labels={'mes': 'Mês', 'feriados': 'Feriados'},
         color_discrete_sequence=[LIGHT_GRAY, BLUE_HIGHLIGHT, BLUE_HIGHLIGHT, LIGHT_GRAY, BLUE_HIGHLIGHT, LIGHT_GRAY, LIGHT_GRAY, LIGHT_GRAY, BLUE_HIGHLIGHT, LIGHT_GRAY]
         )

## Pergunta 3: Quantos feriados em 2024 caem em dias de semana (segunda a sexta-feira)?



In [109]:
import datetime

# Retorna True se date é dia da semana e False caso contrário
def is_weekday(date):
  date_obj = datetime.datetime.strptime(date, '%Y-%m-%d')
  day_of_week = date_obj.weekday()
  return 0 <= day_of_week <= 4  # Monday (0) to Friday (4)

# Adiciona campo weekday a feriados para diferenciar feriados que caem em dia da semana
for feriado in feriados:
  if is_weekday(feriado['date']):
    feriado['weekday'] = True
  else:
    feriado['weekday'] = False

# Cria DataFrame de feriados
df_feriados = pd.DataFrame(feriados)

# Agrupa DataFrame por tipo de dia da semana
df_feriados_weekday = df_feriados.groupby('weekday').size().reset_index(name='feriados')

# Substitui True por 'Dia de semana' e False por 'Fim de semana'
df_feriados_weekday['weekday'] = df_feriados_weekday['weekday'].replace({True: 'Dia de semana', False: 'Fim de semana'})

plot_bar(data=df_feriados_weekday,
         x='weekday',
         y='feriados',
         color='weekday',
         text='feriados',
         title='Feriados em Dias da Semana x Fim de Semana',
         subtitle='Nove feriados caem em dias da semana',
         labels={'weekday': 'Dia da semana', 'feriados': 'Feriados'},
         color_discrete_sequence=[LIGHT_GRAY, BLUE_HIGHLIGHT]
         )

## Pergunta 4: Qual foi a temperatura média em cada mês?
Utilize a Open-Meteo Historical Weather API para obter as temperaturas médias diárias no Rio de Janeiro de 01/01/2024 a 01/08/2024.

In [110]:
# Configura o Open-Meteo API, adiciona cache de consultas e configura tentativas em caso de falha na requisição
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Requisitando dados da API
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": -22.91,
	"longitude": -43.18,
	"start_date": "2024-01-01",
	"end_date": "2024-08-01",
	"daily": ["weather_code", "temperature_2m_mean"]
}
responses = openmeteo.weather_api(url, params=params)

# Armazena os dados da primeira localidade escolhida (Rio de Janeiro)
response = responses[0]

# Armazena dados requisitados (Temperatura)
daily = response.Daily()
daily_weather_code = daily.Variables(0).ValuesAsNumpy()
daily_temperature_2m_mean = daily.Variables(1).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}
daily_data["weather_code"] = daily_weather_code
daily_data["temperature_2m_mean"] = daily_temperature_2m_mean

# Cria DataFrame com as temperaturas médias diárias
df_daily = pd.DataFrame(data=daily_data)

# Plota gráfico de linha das temperaturas médias diárias
fig = px.line(df_daily,
              x='date',
              y='temperature_2m_mean',
              title='Temperatura média diária de 2024 até o mês de Agosto',
              labels={'date': 'Data', 'temperature_2m_mean': 'Temperatura Média'}
              )
fig.update_layout(width=1080)
fig.show()

/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



## Pergunta 5: Qual foi o tempo predominante em cada mês nesse período?
Utilize como referência para o código de tempo (weather_code) o seguinte link: [WMO Code](https://gist.github.com/stellasphere/9490c195ed2b53c707087c8c2db4ec0c).

In [111]:
# Requisitando arquivo JSON de weather_code
url = 'https://gist.githubusercontent.com/stellasphere/9490c195ed2b53c707087c8c2db4ec0c/raw/76b0cb0ef0bfd8a2ec988aa54e30ecd1b483495d/descriptions.json'
resp = requests.get(url)
codes = json.loads(resp.text)
print(codes)

{'0': {'day': {'description': 'Sunny', 'image': 'http://openweathermap.org/img/wn/01d@2x.png'}, 'night': {'description': 'Clear', 'image': 'http://openweathermap.org/img/wn/01n@2x.png'}}, '1': {'day': {'description': 'Mainly Sunny', 'image': 'http://openweathermap.org/img/wn/01d@2x.png'}, 'night': {'description': 'Mainly Clear', 'image': 'http://openweathermap.org/img/wn/01n@2x.png'}}, '2': {'day': {'description': 'Partly Cloudy', 'image': 'http://openweathermap.org/img/wn/02d@2x.png'}, 'night': {'description': 'Partly Cloudy', 'image': 'http://openweathermap.org/img/wn/02n@2x.png'}}, '3': {'day': {'description': 'Cloudy', 'image': 'http://openweathermap.org/img/wn/03d@2x.png'}, 'night': {'description': 'Cloudy', 'image': 'http://openweathermap.org/img/wn/03n@2x.png'}}, '45': {'day': {'description': 'Foggy', 'image': 'http://openweathermap.org/img/wn/50d@2x.png'}, 'night': {'description': 'Foggy', 'image': 'http://openweathermap.org/img/wn/50n@2x.png'}}, '48': {'day': {'description': '

In [112]:
# Cria uma nova coluna 'weather_description' contendo a tradução do weather_code do tempo durante o Dia
df_daily['weather_description'] = df_daily['weather_code'].apply(lambda code: codes[str(int(code))]['day']['description'])

# Dicionário de tradução
translation = {
    'Light Drizzle': 'Garoa Leve',
    'Drizzle': 'Garoa',
    'Sunny': 'Ensolarado',
    'Mainly Sunny': 'Principalmente Ensolarado',
    'Partly Cloudy': 'Parcialmente Nublado',
    'Cloudy': 'Nublado',
    'Heavy Drizzle': 'Garoa Forte',
    'Rain': 'Chuva',
    'Heavy Rain': 'Chuva Forte',
    'Light Rain': 'Chuva Leve'
}

# Traduz a coluna weather_description para Português de acordo com dicionário translation usando apply lambda
df_daily['weather_description'] = df_daily['weather_description'].apply(lambda x: translation[x] if x in translation else x)

In [113]:
# Agrupa por Tempo
grouped_by_weather = df_daily.groupby('weather_description').size().reset_index(name='count')

# Ordena por Tempo
grouped_by_weather = grouped_by_weather.sort_values(by='count', ascending=False)

plot_bar(data=grouped_by_weather,
         x='count',
         y='weather_description',
         color='weather_description',
         text='count',
         title='Distribuição de tipos de Tempo por dia',
         subtitle='O Rio de Janeiro teve predominantemente dias de garoa até agosto de 2024',
         labels={'weather_description': 'Tempo', 'count': 'Quantidade'},
         orientation='h',
         color_discrete_sequence=[BLUE_HIGHLIGHT] + [LIGHT_GRAY] * (len(grouped_by_weather['weather_description']) - 1)
         )

## Pergunta 6: Qual foi o tempo e a temperatura média em cada feriado de 01/01/2024 a 01/08/2024?



In [114]:
# Converte a coluna 'date' de df_feriados e df_daily para o tipo datetime.date
df_feriados['date'] = pd.to_datetime(df_feriados['date']).dt.date
df_daily['date'] = pd.to_datetime(df_daily['date']).dt.date

# Filtra df_daily para apenas os dias de feriados
df_feriados_daily = df_daily.loc[df_daily['date'].isin(df_feriados['date'])]

# Transforma temperaturas para float e arredonda valores para 2 casas decimais
df_feriados_daily['temperature_2m_mean'] = df_feriados_daily['temperature_2m_mean'].astype(float).round(2)

fig = px.bar(df_feriados_daily,
              x='date',
              y='temperature_2m_mean',
              color='weather_description',
              text='temperature_2m_mean',
              title='Temperatura média e Tempo dos feriados de 2024 até o mês de Agosto',
              labels={'temperature_2m_mean': 'Temperatura média', 'date': 'Dia', 'weather_description': 'Tempo'})
fig.update_traces(textposition='outside')
fig.update_xaxes(type='category')
fig.update_layout(width=1080, xaxis={'categoryorder':'category ascending'})
fig.show()

## Pergunta 7: Considere as seguintes suposições:

- O cidadão carioca considera "frio" um dia cuja temperatura média é menor que 20ºC;
- Um feriado bem aproveitado no Rio de Janeiro é aquele em que se pode ir à praia;
- O cidadão carioca só vai à praia quando não está com frio;
- O cidadão carioca também só vai à praia em dias com sol, evitando dias totalmente nublados ou chuvosos (considere weather_code para determinar as condições climáticas).

Houve algum feriado "não aproveitável" em 2024? Se sim, qual(is)?



In [115]:
# Lista os Tempos dos feriados em 2024
tempos_feriados = df_feriados_daily['weather_description'].unique()

# Lista Tempos não aproveitáveis dentre os climas dos feriados (Parcialmente nublado foi considerado aproveitável pois não é TOTALMENTE nublado)
tempos_nao_aproveitaveis = [tempo for tempo in tempos_feriados if tempo not in ['Ensolarado', 'Principalmente Ensolarado', 'Parcialmente Nublado']]

feriados_nao_aproveitaveis = df_feriados_daily.loc[(df_feriados_daily['temperature_2m_mean'] < 20) | (df_feriados_daily['weather_description'].isin(tempos_nao_aproveitaveis))]

# Adiciona coluna aproveitavel ao dataframe df_feriados_daily
df_feriados_daily['aproveitavel'] = ~df_feriados_daily['date'].isin(feriados_nao_aproveitaveis['date'])

fig = px.bar(df_feriados_daily,
              x='date',
              y='temperature_2m_mean',
              color='weather_description',
              text='temperature_2m_mean',
              pattern_shape='aproveitavel',
              pattern_shape_map={True: "", False: "/"},
              title='Feriados por Temperatura média, Aproveitabilidade e Tempo' + '<br><sup>O ano teve 6 feriados não aproveitáveis (listrados). A garoa leve é o que mais impediu os cariocas de aproveitarem os feriados este ano (4 dos 6 feriados)</sup>',
              labels={'temperature_2m_mean': 'Temperatura média', 'date': 'Dia', 'weather_description': 'Tempo'})
fig.update_traces(textposition='outside')
fig.update_xaxes(type='category')
fig.update_layout(width=1080, xaxis={'categoryorder':'category ascending'})
fig.show()

In [116]:
# Une a tabela de feriados com a df_daily_feriados
feriados_climas = df_feriados_daily.merge(df_feriados, on='date', how='left')

# Seleciona apenas feriados não aproveitaveis
df_feriados_nao_aproveitaveis = feriados_climas[feriados_climas['aproveitavel'] == False]

fig = px.bar(df_feriados_nao_aproveitaveis,
              x='localName',
              y='temperature_2m_mean',
              color='weather_description',
              text='temperature_2m_mean',
              title='Feriados Não Aproveitáveis por Temperatura média e Tempo' + '<br><sup>O ano teve 6 feriados não aproveitáveis (listrados). A garoa leve é o que mais impediu os cariocas de aproveitarem os feriados este ano (4 dos 6 feriados)</sup>',
              labels={'temperature_2m_mean': 'Temperatura média', 'localName': 'Nome do Feriado', 'weather_description': 'Tempo'})
fig.update_traces(textposition='outside')
fig.update_xaxes(type='category')
fig.update_layout(width=1080, height=600)
fig.show()

## Pergunta 8: Qual foi o feriado "mais aproveitável" de 2024?
Considere o melhor par tempo e temperatura.

In [117]:
# Seleciona feriados aproveitaveis
df_feriados_aproveitaveis = feriados_climas[feriados_climas['aproveitavel'] == True]

# Ordena df_feriados_aproveitaveis por temperatura média decrescente
df_feriados_aproveitaveis = df_feriados_aproveitaveis.sort_values(by='temperature_2m_mean', ascending=False)

fig = px.bar(df_feriados_aproveitaveis,
            x='localName',
            y='temperature_2m_mean',
            color='weather_description',
            text='temperature_2m_mean',
            pattern_shape='weekday',
            title='Feriados aproveitáveis por Temperatura média e Tempo' + '<br><sup>O feriado mais aproveitável foi a segunda-feira de Carnaval. Um dia de semana, com tempo Principalmente Ensolarado e temperatura média de 30.17 graus</sup>',
            labels={'temperature_2m_mean': 'Temperatura média', 'localName': 'Nome do Feriado', 'weather_description': 'Tempo'})
fig.update_traces(textposition='outside')
fig.update_xaxes(type='category')
fig.update_layout(width=1080, height=600)
fig.show()